In [140]:
import numpy as np
import os
import pandas as pd

In [226]:
class TreeEnsemble():
    def __init__(self, x, y, n_trees, j, w, J, sample_sz, min_leaf=5, M=8, C=5):
        np.random.seed(42)
        self.j, self.w, self.J, self.M, self.C = j, w, J, M, C
        self.x,self.y,self.sample_sz,self.min_leaf = x,y,sample_sz,min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        idxs = np.random.choice(self.x.index,len(self.x))
        xd = self.x.iloc[idxs,:]
        yd = self.y.iloc[idxs,:]
        return DecisionTree(xd, yd, idxs, self.j, self.w, self.J, self.min_leaf, self.M, self.C)
    
    def calc_surv(self,t,St,time):
        for k,tk in enumerate(time):
            if tk >= t : return St[k]
        return 0.0
    
    def aggregate_surv(self,S,times):
        ts = set(np.array(times).flatten())
        surv_prob = {}
        for t in ts:
            surv_prob[t] = np.mean([self.calc_surv(t,S[b],times[b]) for b in range(self.n_trees)])
        return surv_prob
    
    def predict(self, x):
        Hs_lst,Sts_lst,Times_lst = [],[],[]
        for tree in self.trees:
            Hs,Sts,Times = tree.predict(x)
            Hs_lst.append(Hs)
            Sts_lst.append(Sts)
            Times_lst.append(Times)
        surv_probs = []
        for i in range(len(x)):
            surv = self.aggregate_surv([Sts_lst[b][i] for b in range(self.n_trees)],[Times_lst[b][i] for b in range(self.n_trees)])
            surv_probs.append(surv)
        return surv_probs

In [227]:
class DecisionTree():
    def __init__(self, x, y, idxs, j, w, J, min_leaf=5, M=8, C=5):
        self.x, self.y, self.idxs, self.min_leaf, self.M, self.C = x, y, idxs, min_leaf, M, C
        self.j, self.w, self.J = j, w, J
        self.n, self.c = len(idxs), x.shape[1]
#         self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        # self.var_idx, self.split, self.score --> contain split info for this node.
        
    def terminal_node_estimations(self):
        times = sorted(list(set(self.y[self.idxs,'futime'])))
        m = len(times)
        N,D,Yt = np.zeros((self.J+1,m)),np.zeros((self.J+1,m)),np.zeros((m,))
        for t in times:
            for i in self.idxs:
                if self.y.loc[i,'futime'] >= t : Yt[t] += 1
                if self.y.loc[i,'futime'] <= t :
                    N[self.y.loc[i,'status']][t] += 1
                if self.y.loc[i,'futime'] == t :
                    D[self.y.loc[i,'status']][t] += 1
        Dt = np.sum(D,0)
        Nt = np.sum(N,0)
        ### Nelson–Aalen estimator for the cumulative event-specific hazard function H
        ###  Kaplan–Meier estimator for the event-free survival function St
        H,St = D/Yt, 1 - Dt/Yt
        for t in range(1,m):
            H[:,t] += H[:,t-1] 
            St[t] *= St[t-1]
            
        self.H, self.St,self.times = H,St,times
        
    def find_varsplit(self):
        if len(set(self.idxs)) <= self.min_leaf :
            self.terminal_node_estimations()
            return
        for var_idx in np.random.choice(self.c,self.M):
            for c in np.random.choice(self.x.iloc[:,var_idx],self.C):
                lr = self.calc_logrank(var_idx,c,self.j,self.w[self.j])
                if lr > self.score:
                    self.score, self.var_idx, self.split = lr, var_idx, c
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        self.left = DecisionTree(self.x, self.y, self.idxs[lhs], self.j, self.w, self.J, self.min_leaf, self.M, self.C)
        self.right = DecisionTree(self.x, self.y, self.idxs[rhs], self.j, self.w, self.J, self.min_leaf, self.M, self.C)
        
    def calc_logrank(self,var_idx, c, j, wj):
        times = sorted(list(set(self.y.loc[self.idxs,'futime'])))
        m = len(times)
        Y_l,Y_r,dj_l,dj_r = np.zeros((m,)),np.zeros((m,)),np.zeros((m,)),np.zeros((m,))
        for t in times:
            for i in self.idxs:
                print(i,self.y.loc[i,'futime'],t)
                if self.y.loc[i,'futime'] >= t :
                    if self.x.iloc[i,var_idx] <= c : Y_l[t] += 1
                    else : Y_r[t] += 1
                if self.y.loc[i,'futime'] == t and self.y.loc[i,'status'] == j :
                    if self.x.iloc[i,var_idx] <= c : dj_l[t] += 1
                    else : dj_r[t] += 1
        Y = Y_l + Y_r
        dj = dj_l + dj_r
        Wj = np.array([wj]*m)
        
        sigj = np.sum((Wj**2)*dj*(Y_l/Y)*(1 - Y_l/Y)*((Y-dj)/(Y-1)))
        Lj = np.sum(Wj*(dj_l - dj*Y_l/Y))/sigj
        
        return abs(Lj)
        
    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf')
    
    def __repr__(self):
        s = f'n: {self.n}; val:{self.val}'
        if not self.is_leaf:
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s

    def predict(self, x): ## x --> numpy array
        Hs,Sts,Times = [],[],[]
        for xi in x:
            H,St,times = self.predict_row(xi)
            Hs.append(H)
            Sts.append(St)
            Times.append(times)
        return Hs,Sts,Times

    def predict_row(self, xi):
        if self.is_leaf: return self.H,self.St,self.times
        tree = self.left if xi[self.var_idx]<=self.split else self.right
        return tree.predict_row(xi)

In [190]:
# id       = case number
# futime   = number of days between registration and the earlier of death,
#            transplantion, or study analysis time in July, 1986
# status   = 0=alive(censored), 1=liver transplant, 2=dead
# drug     = 1= D-penicillamine, 2=placebo
# age      = age in days
# sex      = 0=male, 1=female
# ascites  = presence of ascites: 0=no 1=yes
# hepato   = presence of hepatomegaly 0=no 1=yes
# spiders  = presence of spiders 0=no 1=yes
# edema    = presence of edema 0=no edema and no diuretic therapy for edema;
#           .5 = edema present without diuretics, or edema resolved by diuretics;
#            1 = edema despite diuretic therapy
# bili     = serum bilirubin in mg/dl
# chol     = serum cholesterol in mg/dl
# albumin  = albumin in gm/dl
# copper   = urine copper in ug/day
# alk_phos = alkaline phosphatase in U/liter
# sgot     = SGOT in U/ml
# trig     = triglicerides in mg/dl
# platelet = platelets per cubic ml/1000
# protime  = prothrombin time in seconds
# stage    = histologic stage of disease

In [207]:
cols = ['id','futime','status','drug','age','sex','ascites','hepato','spiders','edema','bili','chol','albumin','copper','alk_phos','sgot','trig','platelet','protime','stage']

In [208]:
df = pd.DataFrame(columns=cols)
with open('pbc.dat.txt','r') as f:
    for line in f.readlines():
        row = [float(n) if n!='.' else np.nan for n in line.split()]
        df.loc[len(df.index)] = row
X,y = df.drop(columns=['id','futime','status']),df[['futime','status']]

In [231]:
X,y = X.interpolate(),y.interpolate()

In [232]:
y

,futime,status
0,400.0,2.0
1,4500.0,0.0
2,1012.0,2.0
3,1925.0,2.0
4,1504.0,1.0
...,...,...
413,681.0,2.0
414,1103.0,0.0
415,1055.0,0.0
416,691.0,0.0


In [233]:
forest = TreeEnsemble(x=X, y=y, n_trees=1000, j=1, w={1:1,2:0}, J=2, sample_sz=len(X), min_leaf=5, M=8, C=5)

102 102    110.0
102    110.0
Name: futime, dtype: float64 41.0


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [230]:
np.nan > 2

False